In [1]:
import sys
import os
gems_tco_path = "/Users/joonwonlee/Documents/GEMS_TCO-1/src"
sys.path.append(gems_tco_path)

# Data manipulation and analysis
import pandas as pd
import numpy as np
import pickle 

import GEMS_TCO
from GEMS_TCO import kernels 
from GEMS_TCO import orderings as _orderings
from GEMS_TCO import load_data

import torch
from collections import defaultdict

import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import copy



In [2]:
lat_lon_resolution = [20,20]
mm_cond_number = 10
params= [20, 8.25, 5.25, 0.2, 0.5, 5]
idx_for_datamap= [0,8]

In [10]:
# Load the one dictionary to set spaital coordinates
# filepath = "C:/Users/joonw/TCO/GEMS_data/data_2023/sparse_cen_map23_01.pkl"
filepath = "/Users/joonwonlee/Documents/GEMS_DATA/pickle_2023/coarse_cen_map23_01.pkl"
with open(filepath, 'rb') as pickle_file:
    coarse_dict_24_1 = pickle.load(pickle_file)

sample_df = coarse_dict_24_1['y23m01day01_hm02:12']
sample_key = coarse_dict_24_1.get('y23m01day01_hm02:12')
if sample_key is None:
    print("Key 'y23m01day01_hm02:12' not found in the dictionary.")

# { (20,20):(5,1), (5,5):(20,40) }
rho_lat = lat_lon_resolution[0]          
rho_lon = lat_lon_resolution[1]
lat_n = sample_df['Latitude'].unique()[::rho_lat]
lon_n = sample_df['Longitude'].unique()[::rho_lon]

lat_number = len(lat_n)
lon_number = len(lon_n)

# Set spatial coordinates for each dataset
coarse_dicts = {}

years = ['2024']
for year in years:
    for month in range(7, 8):  # Iterate over all months
        # filepath = f"C:/Users/joonw/TCO/GEMS_data/data_{year}/sparse_cen_map{year[2:]}_{month:02d}.pkl"
        filepath = f"/Users/joonwonlee/Documents/GEMS_DATA/pickle_{year}/coarse_cen_map{year[2:]}_{month:02d}.pkl"
        with open(filepath, 'rb') as pickle_file:
            loaded_map = pickle.load(pickle_file)
            for key in loaded_map:
                tmp_df = loaded_map[key]
                coarse_filter = (tmp_df['Latitude'].isin(lat_n)) & (tmp_df['Longitude'].isin(lon_n))
                coarse_dicts[f"{year}_{month:02d}_{key}"] = tmp_df[coarse_filter].reset_index(drop=True)

key_idx = list( coarse_dicts.keys() )
if not key_idx:
    raise ValueError("coarse_dicts is empty")

# extract first hour data because all data shares the same spatial grid
data_for_coord = coarse_dicts[key_idx[0]]
x1 = data_for_coord['Longitude'].values
y1 = data_for_coord['Latitude'].values 
coords1 = np.stack((x1, y1), axis=-1)


# instance = orbitmap.MakeOrbitdata(data_for_coord, lat_s=5, lat_e=10, lon_s=110, lon_e=120)
# s_dist = cdist(coords1, coords1, 'euclidean')
# ord_mm, _ = instance.maxmin_naive(s_dist, 0)

ord_mm = _orderings.maxmin_cpp(coords1)
data_for_coord = data_for_coord.iloc[ord_mm].reset_index(drop=True)
coords1_reordered = np.stack((data_for_coord['Longitude'].values, data_for_coord['Latitude'].values), axis=-1)
# nns_map = instance.find_nns_naive(locs=coords1_reordered, dist_fun='euclidean', max_nn=mm_cond_number)
nns_map=_orderings.find_nns_l2(locs= coords1_reordered  ,max_nn = mm_cond_number)


analysis_data_map = {}
for i in range(idx_for_datamap[0],idx_for_datamap[1]):
    tmp = coarse_dicts[key_idx[i]].copy()
    tmp['Hours_elapsed'] = np.round(tmp['Hours_elapsed']-477700)

    tmp = tmp.iloc[ord_mm, :4].to_numpy()
    tmp = torch.from_numpy(tmp).double()  # Convert NumPy to Tensor
    # tmp = tmp.clone().detach().requires_grad_(True)  # Enable gradients
    
    analysis_data_map[key_idx[i]] = tmp

aggregated_data = pd.DataFrame()
for i in range(idx_for_datamap[0],idx_for_datamap[1]):
    tmp = coarse_dicts[key_idx[i]].copy()
    tmp['Hours_elapsed'] = np.round(tmp['Hours_elapsed']-477700)
    tmp = tmp.iloc[ord_mm].reset_index(drop=True)  
    aggregated_data = pd.concat((aggregated_data, tmp), axis=0)

aggregated_data = aggregated_data.iloc[:, :4].to_numpy()


aggregated_data = torch.from_numpy(aggregated_data).double()  # Convert NumPy to Tensor
# aggregated_np = aggregated_np.clone().detach().requires_grad_(True)  # Enable gradients




In [15]:
result = {}
nheads= 100

for day in range(1):
    idx_for_datamap= [8*day,8*(day+1)]
    analysis_data_map = {}
    for i in range(idx_for_datamap[0],idx_for_datamap[1]):
        tmp = coarse_dicts[key_idx[i]].copy()
        tmp['Hours_elapsed'] = np.round(tmp['Hours_elapsed']-477700)

        tmp = tmp.iloc[ord_mm, :4].to_numpy()
        tmp = torch.from_numpy(tmp).double()  # Convert NumPy to Tensor
        # tmp = tmp.clone().detach().requires_grad_(True)  # Enable gradients
        
        analysis_data_map[key_idx[i]] = tmp
    aggregated_data = pd.DataFrame()
    for i in range(idx_for_datamap[0],idx_for_datamap[1]):
        tmp = coarse_dicts[key_idx[i]].copy()
        tmp['Hours_elapsed'] = np.round(tmp['Hours_elapsed']-477700)
        tmp = tmp.iloc[ord_mm].reset_index(drop=True)  
        aggregated_data = pd.concat((aggregated_data, tmp), axis=0)
    
    aggregated_data = aggregated_data.iloc[:, :4].to_numpy()

    aggregated_data = torch.from_numpy(aggregated_data).double()  # Convert NumPy to Tensor

    params = [24.42, 1.92, 1.92, 0.001, -0.045, 0.237, 3.34]
    # params = [24.42, 10, 10, 0.001, -0.045, 0.4, 3.34]
    params = torch.tensor(params, requires_grad=True)

    params = [23.09, 2.24, 3.2927, -0.107, 0.1659, 0.1612, 1.6049]
    params = torch.tensor(params, dtype=torch.float64, requires_grad=True)

    torch_smooth = torch.tensor(0.5, dtype=torch.float32)


    instance = kernels.model_fitting(
        smooth= 1.0,
        input_map=analysis_data_map,
        aggregated_data=aggregated_data,
        nns_map=nns_map,
        mm_cond_number=mm_cond_number,
        nheads= nheads
    )

    # optimizer = optim.Adam([params], lr=0.01)  # For Adam
    optimizer, scheduler = instance.optimizer_fun(params, lr=0.03, betas=(0.9, 0.99), eps=1e-8, step_size=100, gamma=0.9)    
    out = instance.run_full(params, optimizer,scheduler, instance.matern_cov_anisotropy_kv, epochs=1000)
    result[day+1] = out
        

Epoch 1, Gradients: [   -7.12808734  -422.33365962  -633.95397821   223.63099248
 -1242.94883333  4342.84355591  -168.82206927]
 Loss: 876.6676864038084, Parameters: [23.09    2.24    3.2927 -0.107   0.1659  0.1612  1.6049]
Epoch 11, Gradients: [-7.50180789e+00 -1.93382965e+03 -2.87630169e+03  4.70085275e+05
  1.19224082e+05  1.60005008e+04 -6.46036130e+02]
 Loss: 1812.9022785826803, Parameters: [2.33916678e+01 2.52331985e+00 3.49616626e+00 9.03205155e-03
 4.20710422e-01 4.26896121e-05 1.90835434e+00]
Epoch 21, Gradients: [-5.79605970e+00 -1.39373063e+02 -1.30209721e+03  8.04208382e+04
  1.79567481e+05 -9.41801690e+05 -3.94618846e+02]
 Loss: 1440.1109794636309, Parameters: [ 2.36918819e+01  2.72822235e+00  3.75605044e+00  2.99442658e-03
  3.52090921e-01 -4.18449028e-03  2.22190710e+00]
Epoch 31, Gradients: [-4.26444857e+00 -2.19147552e+02 -1.66695563e+02  9.85280900e+02
 -5.61437546e+03  2.16664800e+04 -1.17243597e+02]
 Loss: 858.2621456524253, Parameters: [2.39557869e+01 2.83659460e+0

### Save estimates in to pickle fime

In [3]:
isinstance(result, dict)
import os
# Save pickle
output_filename = f"estimation_1250_july24.pkl"

# base_path = "/home/jl2815/tco/data/pickle_data"
output_path = "/Users/joonwonlee/Documents/"
output_filepath = os.path.join(output_path, output_filename)
with open(output_filepath, 'wb') as pickle_file:
    pickle.dump(result, pickle_file)

input_filepath = output_filepath
# Load pickle
with open(input_filepath, 'rb') as pickle_file:
    loaded_map = pickle.load(pickle_file)

loaded_map

NameError: name 'result' is not defined

## load estimates from amarel and make it into pd data frame

10x10 full     takes 22 seconds (epochs 444)

vecchia log
10 by 10 fix 300 epochs (2m 7.2)  
original 2m 7.2  25 3 2.8 -0.04 -0.04 0.0066 4.3

if dont use cache   300 epochs fixed 5m 35.3
25 3 2.84 -0.04 -0.04 0.0066 4.3

In [ ]:

input_path = "/Users/joonwonlee/Documents/GEMS_TCO-1/Exercises/st_model/estimates"

# input_filename = "estimation_50_july24.pkl"
# input_filename = "estimation_200_july24.pkl"
input_filename = "full_estimation_1250_july24.pkl"
input_filepath = os.path.join(input_path, input_filename)
# Load pickle
with open(input_filepath, 'rb') as pickle_file:
    amarel_map1250= pickle.load(pickle_file)

# Assuming df_1250 is your DataFrame
df_1250 = pd.DataFrame()
for key in amarel_map1250:
    tmp = pd.DataFrame(amarel_map1250[key][0].reshape(1, -1), columns=['sigmasq', 'range_lat', 'range_lon', 'advec_lat', 'advec_lon', 'beta', 'nugget'])
    tmp['loss'] = amarel_map1250[key][1]
    df_1250 = pd.concat((df_1250, tmp), axis=0)

# Generate date range
date_range = pd.date_range(start='07-01-24', end='07-31-24')

# Ensure the number of dates matches the number of rows in df_1250
if len(date_range) == len(df_1250):
    df_1250.index = date_range
else:
    print("The number of dates does not match the number of rows in the DataFrame.")


print(df_1250)
df = df_1250
# Save DataFrame to CSV
output_filename = 'estimation_1250_july24.csv'
output_csv_path = os.path.join(input_path, output_filename)
df.to_csv(output_csv_path, index=False)

,sigmasq,range_lat,range_lon,advec_lat,advec_lon,beta,nugget,loss
2024-07-01,24.793444,1.584529,1.718248,0.009089,-0.107299,0.131038,2.717239,14068.529297
2024-07-02,24.424301,1.997055,1.942683,0.043588,-0.072679,0.137124,1.513148,12357.715820
2024-07-03,26.009497,1.215236,1.558868,0.023392,-0.150548,0.199850,2.890678,14948.140625
2024-07-04,24.701347,1.612308,1.822960,-0.164069,-0.237443,0.131595,3.636499,14786.204102
2024-07-05,22.598671,2.901185,3.722327,-0.011729,-0.152072,0.072866,2.397249,12096.261719
2024-07-06,25.594908,1.702692,2.255174,0.017462,-0.158125,0.098684,3.850205,14690.248047
2024-07-07,26.030510,1.261084,2.831952,0.054831,-0.343255,0.103045,4.596346,15342.459961
2024-07-08,26.043682,0.995279,1.629503,-0.019824,-0.411626,0.164296,2.751402,14857.195312
2024-07-09,24.052071,1.377774,2.357721,0.021439,-0.220316,0.142847,1.675457,12666.991211
2024-07-10,25.766109,1.392051,2.358171,0.026684,-0.077366,0.150648,3.821218,14987.769531
